# Building end to end data engineering project with python

# we will be working with data, building a data model then deploying the data model into the database. 

In [340]:
import psycopg2
import pandas as pd

In [341]:
def create_database():
    # connect to default database
    # conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE accounts")
    cur.execute("CREATE DATABASE accounts")
    
    # close connection to default database
    conn.close()
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=root")
    cur = conn.cursor()
    
    return cur, conn

In [342]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [343]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [344]:
AccountsCountry = pd.read_csv("data/Wealth-AccountsCountry.csv")

In [345]:
AccountsCountry.head()

,Country Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


In [346]:
AccountsCountry_clean = AccountsCountry[['Country Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [347]:
AccountsCountry_clean.head()

,Country Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


In [348]:
AccountsData = pd.read_csv("data/Wealth-AccountsData.csv")

In [349]:
AccountsData.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1995,1996,1997,1998,1999,2000,2001,2002,2005,2010,2014
0,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,"14,072.79","13,708.99","11,794.10","12,399.48","13,573.26","14,118.13","15,564.46","15,906.19","18,072.30","23,362.86","23,362.86"
1,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,"3,468.86","3,355.55","2,869.00","3,013.39","3,259.89","3,424.01","3,783.82","3,906.70","2,222.12","2,889.15","2,889.15"
2,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,"6,507.93","6,353.74","5,466.69","5,901.07","6,492.73","6,872.02","7,607.77","7,820.39","11,689.35","15,789.92","15,789.92"
3,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,"4,893.01","4,737.86","4,059.32","4,191.45","4,536.97","4,695.07","5,169.88","5,298.90","2,886.74","3,613.61","3,613.61"
4,Albania,ALB,"Human capital per capita, male (constant 2018 ...",NW.HCA.MALE.PC,"9,179.78","8,971.14","7,734.78","8,208.04","9,036.29","9,423.06","10,394.58","10,607.29","15,185.56","19,749.26","19,749.26"


In [350]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2005',
       '2010', '2014'],
      dtype='object')

In [351]:
AccountsData.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1995,1996,1997,1998,1999,2000,2001,2002,2005,2010,2014
0,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,"14,072.79","13,708.99","11,794.10","12,399.48","13,573.26","14,118.13","15,564.46","15,906.19","18,072.30","23,362.86","23,362.86"
1,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,"3,468.86","3,355.55","2,869.00","3,013.39","3,259.89","3,424.01","3,783.82","3,906.70","2,222.12","2,889.15","2,889.15"
2,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,"6,507.93","6,353.74","5,466.69","5,901.07","6,492.73","6,872.02","7,607.77","7,820.39","11,689.35","15,789.92","15,789.92"
3,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,"4,893.01","4,737.86","4,059.32","4,191.45","4,536.97","4,695.07","5,169.88","5,298.90","2,886.74","3,613.61","3,613.61"
4,Albania,ALB,"Human capital per capita, male (constant 2018 ...",NW.HCA.MALE.PC,"9,179.78","8,971.14","7,734.78","8,208.04","9,036.29","9,423.06","10,394.58","10,607.29","15,185.56","19,749.26","19,749.26"


In [380]:
AccountsData[["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2005", "2010", "2014"]] = AccountsData[["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2005", "2010", "2014"]].infer_objects()

In [381]:
AccountsSeries = pd.read_csv("data/Wealth-AccountsSeries.csv")

In [382]:
AccountsSeries.columns

Index(['Series Code', 'Indicator Name', 'Short definition', 'Long definition',
       'Source', 'Topic', 'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [383]:
AccountsSeries = AccountsSeries[['Series Code', 'Topic', 'Indicator Name', 'Short definition']]

In [384]:
AccountsSeries.head()

,Series Code,Topic,Indicator Name,Short definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),NaN
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),NaN
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",NaN
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",NaN
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",NaN


In [387]:
cur, conn = create_database()

In [388]:
accounts_country_table_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(
country_code VARCHAR PRIMARY KEY,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
)""")
cur.execute(accounts_country_table_create)
conn.commit()

In [389]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR,
year_1995 numeric,
year_2000 numeric,
year_2005 numeric,
year_2010 numeric,
year_2014 numeric
)""")
cur.execute(accounts_data_table_create)
conn.commit()

In [390]:
accounts_series_table_create = ("""CREATE TABLE IF NOT EXISTS accountsseries(
series_code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR,
short_definition VARCHAR
)""")
cur.execute(accounts_series_table_create)
conn.commit()

In [391]:
accounts_country_table_insert = ("""INSERT INTO accountscountry(
country_code,
short_name,
table_name,
long_name,
currency_unit)
VALUES (%s, %s, %s, %s, %s)
""")

In [392]:
for i, row in AccountsCountry_clean.iterrows():
    cur.execute(accounts_country_table_insert, list(row))
conn.commit()

In [393]:
accounts_data_table_insert = ("""INSERT INTO accountsdata(
country_name,
country_code,
indicator_name,
indicator_code,
year_1995,
year_2000,
year_2005,
year_2010,
year_2014)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [395]:
for i, row in AccountsData.iterrows():
    cur.execute(accounts_data_table_insert, list(row))
conn.commit()

TypeError: not all arguments converted during string formatting

In [396]:
accounts_series_table_insert = ("""INSERT INTO accountsseries(
series_code,
topic,
indicator_name,
short_definition)
VALUES (%s, %s, %s, %s)
""")

In [397]:
for i, row in AccountsSeries.iterrows():
    cur.execute(accounts_series_table_insert, list(row))
conn.commit()

In [386]:
cur.close()
conn.close()